In [1]:
import os
import geopandas as gpd
import pandas as pd
import folium
import webbrowser

Read geospatial parquet files contain bank desert status and bank locations to GeoPandas DataFrames. Read Census Data to Pandas Dataframe.

In [2]:
# go up a directory level
os.chdir('..')

# bank desert status
bds_gdf = gpd.read_parquet('exploratory_data_analysis/shp_with_BDS.parquet')

# census data
acs5 = pd.read_parquet('data_preprocessing/acs5_final.parquet')

# bank locations
banks_gdf = gpd.read_parquet('data_preprocessing/fdic_ncua_gdf_final.parquet')

# rest to cwd
os.chdir('data_visualization')

In [3]:
# convert GEOIDs to string for merge
bds_gdf["GEOID"] = bds_gdf["GEOID"].astype("string")
acs5["GEOID"] = acs5["GEOID"].astype("string")

In [4]:
# filter for California
bds_gdf = bds_gdf[bds_gdf['STATEFP'] == '06']
banks_gdf = banks_gdf[banks_gdf['State'] == 'CA']

In [5]:
# change the projection
bds_gdf = bds_gdf.to_crs(epsg=4326)

In [6]:
# merge bank desert location and census datasets
gdf = bds_gdf.merge(acs5, on='GEOID')

In [7]:
# convert bank desert status to numeric
# 1 = not a bank desert, 2 = potential, 3 = bank desert
gdf['bank_desert_type'] = gdf.apply(lambda x: 1 if x.bank_desert_status == 'not a bank desert' else (2 if x.bank_desert_status == 'potential bank desert' else (3 if x.bank_desert_status == 'bank desert' else 0)), axis=1)

# remove % from Majority Race
gdf['Majority Race'] = gdf['Majority Race'].astype("string").str.replace('%','')


In [8]:
gdf_final = gdf.loc[:,['GEOID','geometry','Bank Desert Status','bank_desert_type','Community Type_x','House Units','Income','Poverty%','Employment%','Majority Race']]

In [9]:
# create a basemap
m = folium.Map(location=[37.1661, -119.4494], zoom_start=5) # centered on central California 
#m = folium.Map(location=[39.8283, -98.5795]) # center on Kansas

In [10]:
# set bins for bank desert status
bins = [0,1,2,3,4]

In [11]:
# add a choropleth layer
folium.Choropleth(
    geo_data=gdf_final, 
    name="Bank Deserts by Census Tract",
    data=gdf_final,
    columns=["GEOID", "bank_desert_type"], 
    key_on="feature.properties.GEOID", 
    fill_color="YlGnBu", # color scale: yellow, green, blue
    bins=bins, # set number of color bins
    reset=True, # reset bins
    fill_opacity=0.7,
    line_opacity=0, # transparent border
    legend_name="Bank Desert Status"
).add_to(m)

In [12]:
tooltip = folium.GeoJsonTooltip(
    fields=["GEOID", "Bank Desert Status", "Community Type_x", "House Units", "Income", "Majority Race"],  # Columns to display in tooltip
    aliases=["Census Tract:", "Bank Desert Status:", "Community Type:", "Housing Units:", "Median Income:", "Racial Majority:"],  # Labels for the tooltip
    localize=True, # format numbers properly
    sticky=False, # tooltip follows the cursor
    labels=True, # show field labels
    style="background-color: white; color: black; font-weight: bold;"  # Tooltip style
)

# add tooltip to the GeoJSON layer
folium.GeoJson(
    gdf,
    tooltip=tooltip,
    style_function=lambda feature: {
        "fillColor": "transparent",  
        "color": "transparent", # transparent outline
        "weight": 0.3,  # thin borders
        "fillOpacity": 0.7
    }
).add_to(m)

In [13]:
# create markers for bank locations
for idx, row in banks_gdf.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=1, # size of the marker
        color="black",
        fill=True,
        fill_color="black",
        fill_opacity=0.8,
        popup=folium.Popup(row["Bank Name"], parse_html=True)  # Show name on click
    ).add_to(m)

In [14]:
# display final map
# m

In [15]:
# save the map as an HTML file
m.save("bank_deserts_map.html")

In [16]:
webbrowser.open("bank_deserts_map.html")

True